In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

In [ ]:
from sklearn.datasets import fetch_california_housing

ds = fetch_california_housing()
ds.DESCR

In [ ]:
X = ds.data
y = ds.target

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
help(Sequential)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation

model = Sequential()

model.add(Input(shape=(8,)))           # 8 features, so 8 inputs
model.add(Dense(8, activation='relu')) # first layer
model.add(Dense(4, activation='relu')) # hidden layer
model.add(Dense(1))                    # output layer, 1 output (predicted value)

model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=50, verbose=1)

In [ ]:
loss = model.history.history['loss']

In [ ]:
sns.lineplot(x=range(len(loss)), y=loss)
plt.title("Cost per Epoch");

In [ ]:
model.evaluate(X_test,y_test,verbose=0) # gives MSE on test data

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions_df = pd.DataFrame(y_test, columns=['Actual Values'])
predictions = pd.Series(predictions.reshape(predictions.shape[0],))
predictions_df = pd.concat([predictions_df, predictions], axis=1)
predictions_df.columns = ['Actual Values', 'Predictions']
predictions_df.head()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(x='Actual Values', y='Predictions', data=predictions_df, ax=ax)
ax.set_ylim(0,5)
ax.set_yticks(range(0,5))

In [ ]:
predictions_df['Error'] = predictions_df['Actual Values'] - predictions_df['Predictions']
sns.histplot(predictions_df['Error'], bins=100, kde=True)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions_df['Actual Values'], predictions_df['Predictions']) # should match MSE given by model.evaluate()

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictions_df['Actual Values'], predictions_df['Predictions'])